In [1]:
# 参考文献
# https://zenn.dev/schnell/articles/c5d9bb70490305

# %%capture output

import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

ja_models = ["nlp-waseda/roberta-base-japanese","nlp-waseda/roberta-large-japanese","nlp-waseda/roberta-large-japanese-seq512"]
ja_model = ja_models[1]

tokenizer =    AutoTokenizer.from_pretrained(ja_model)
model = AutoModelForMaskedLM.from_pretrained(ja_model)

2023-02-10 14:29:16.180160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 14:29:18.317592: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-10 14:29:18.317694: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-10 14:29:18.317705: W tensorfl

In [2]:
# maskされたトークン位置を取得する
def get_masked_index(encoding):
    for idx, id in enumerate(encoding.input_ids.squeeze(0)):
        if id == tokenizer.mask_token_id:
            return idx

# 入力系列の処理
sentence = '日本 語 の 自然 言語 処理 は [MASK] 。'
encoding = tokenizer(sentence, return_tensors='pt')

# 予測
pred = model(**encoding)

# 後処理(取得した予測分布から、maskトークンの上位k件の予測結果を取得)
masked_idx = get_masked_index(encoding)
target_probs = pred.logits.squeeze(0)[masked_idx]

# 上位k件の予測結果をのindexを取得
top_k = torch.topk(target_probs, k=5).indices
for id in top_k:
    print(sentence.replace("[MASK]", f"'{tokenizer.decode(id)}'"))
# 日本 語 の 自然 言語 処理 は '得意' 。
# 日本 語 の 自然 言語 処理 は 'できる' 。
# 日本 語 の 自然 言語 処理 は '専門' 。
# 日本 語 の 自然 言語 処理 は 'ない' 。
# 日本 語 の 自然 言語 処理 は '苦手' 。

日本 語 の 自然 言語 処理 は 'こちら' 。
日本 語 の 自然 言語 処理 は '難しい' 。
日本 語 の 自然 言語 処理 は '可能です' 。
日本 語 の 自然 言語 処理 は '困難である' 。
日本 語 の 自然 言語 処理 は 'これ' 。


In [ ]:
# output.show()


In [7]:
# !pip show tokenizers
!pip show transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Name: tokenizers
Version: 0.13.2
Summary: Fast and Customizable Tokenizers
Home-page: https://github.com/huggingface/tokenizers
Author: Anthony MOI
Author-email: anthony@huggingface.co
License: Apache License 2.0
Location: /home/nb-user/.local/lib/python3.8/site-packages
Requires: 
Required-by: transformers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Name: transformers
Version: 4.26.1
Summary: